In [2]:
import pandas as pd
import numpy as np
import numpy.random as nprand
import math

In [3]:
df = pd.read_csv("data/train.csv")
dfn = df.to_numpy()
dfn = np.hsplit(dfn, [1])
y = dfn[0]
x = dfn[1]
print(f"input shape:  {x.shape}")
print(f"output shape: {y.shape}")

input shape:  (42000, 784)
output shape: (42000, 1)


In [4]:
# normalize our inputs
row_sums = x.sum(axis=1)
# could use np.atleast_2d(row_sums) to ensure that row_sums is (3,1) and not (3,)
x = x / row_sums[:, np.newaxis]

# change our outputs to be vectorized
def expected_transform(expected):
    new_arr = np.zeros((len(expected), 10))
    for i in range(len(expected)):
        new_arr[i][expected[i][0]] = 1

    return new_arr

y = expected_transform(y)
print(f"output shape: {y.shape}")

output shape: (42000, 10)


In [44]:
# an implementation of sigmoid function
#
# Thanks to source:
# https://stackoverflow.com/questions/3985619/how-to-calculate-a-logistic-sigmoid-function-in-python
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# assumes x is already sigmoid(x)
def sigmoid_derivative(x):
    return sigmoid(x) * sigmoid(1 - x)

# an implementation of a ReLU (rectified linear unit) 
# function
#
# Thanks to source: 
# https://stackoverflow.com/questions/32109319/how-to-implement-the-relu-function-in-numpy
#
def relu(x):
    return np.maximum(x, 0)

# an implementation of the ReLU derivative
#
# Thanks to source:
# https://stackoverflow.com/questions/46411180/implement-relu-derivative-in-python-numpy
def relu_derivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

# an implementation of the softmax function
#
# Thanks to source:
# https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()

def softmax_derivative(x):
    pass

softmax_test = np.array([1, 0, 5, 7, 9])


[2.90667863e-04 1.06930731e-04 1.58699276e-02 1.17263785e-01
 8.66468688e-01]
1.0


In [19]:
# Generates a distribution of random numbers in some range,
# specified by the init_type parameter.
#   n is the number of input nodes 
#   m is the number of output nodes.
#   U is the uniform distribution. 
#   G is the gaussian or normal distribution.
#
# Thanks to source: 
# https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/
# 
# Initialization type (init_type) can be:
#
# Good for Sigmoid and Tanh activation functions.
# "xavier"
# Xavier Glorot uses the formula: 
#       weight = U[-(1/sqrt(n)), 1/sqrt(n)]
#
# "nxavier"
# Normalized Xavier Glorot uses the formula:
#       weight = U[-(sqrt(6)/sqrt(n+m)), sqrt(6)/sqrt(n+m)]
# 
# Good for ReLU activation funtions.
# "he"
# Kaiming He uses the formula:
#       weight = G(0.0, sqrt(2/n))
#
#
# 
def initialize_layer_weights(n, m, init_type="xavier"):
    if "xavier" in init_type:
        numbers = nprand.rand(m, n)
        if init_type == "xavier":
            lower, upper = -(1.0 / math.sqrt(n)), (1.0 / math.sqrt(n))
        else:
            lower, upper = -(math.sqrt(6.0) / math.sqrt(n + m)), (math.sqrt(6.0) / math.sqrt(n + m))
        scaled = lower + numbers * (upper - lower)
    else:
        std = math.sqrt(2.0 / n)
        numbers = nprand.randn(m, n)
        scaled = numbers * std

    return scaled

layers = [x.shape[1], 128, 10]

# initialize our layer 1 and layer 2 weights
# layer 1 uses a ReLU function, so use "he" init type
# layer 2 uses a sigmoid function, so use the xavier or nxavier type
l1_w = initialize_layer_weights(layers[0], layers[1], "he")
l2_w = initialize_layer_weights(layers[1], layers[2], "nxavier")

# initialize our bias to 0 for all layers
l1_b = np.zeros((layers[1], 1))
l2_b = np.zeros((layers[2], 1))

print(f"Layer 1 weights shape {l1_w.shape}")
print(f"Layer 2 weights shape {l2_w.shape}")
print(f"Layer 1 bias shape    {l1_b.shape}")
print(f"Layer 2 bias shape    {l2_b.shape}")

Layer 1 weights shape (128, 784)
Layer 2 weights shape (10, 128)
Layer 1 bias shape    (128, 1)
Layer 2 bias shape    (10, 1)


In [20]:
# forward propegate
mini_batch_x = x[0:20].T
mini_batch_y = y[0:20].T
z1 = l1_w.dot(mini_batch_x) + l1_b
a1 = relu(z1)
z2 = l2_w.dot(a1) + l2_b
a2 = sigmoid(z2)
print(a2.shape)

(10, 20)


In [21]:
da2 = (a2 - mini_batch_y)
dz2 = da2 * sigmoid_derivative(z2)
nabla_l2_b = np.sum(dz2, axis=1, keepdims=True)
nabla_l2_w = np.dot(dz2, a1.T)

print(nabla_l2_w.shape)
print(nabla_l2_b.shape)


(10, 128)
(10, 1)


In [22]:
da1 = np.dot(l2_w.T, dz2)
dz1 = da1 * relu_derivative(a1)
nabla_l1_b = np.sum(dz1, axis=1, keepdims=True)
nabla_l1_w = np.dot(dz1, mini_batch_x.T)
print(nabla_l1_w.shape)
print(nabla_l1_b.shape)

(128, 784)
(128, 1)


In [23]:
learning_rate = 0.2

l1_w = l1_w - learning_rate * nabla_l1_w
l2_w = l2_w - learning_rate * nabla_l2_w
l1_b = l1_b - learning_rate * nabla_l1_b
l2_b = l2_b - learning_rate * nabla_l2_b


In [41]:
# put it all together:

mini_batch_size = 47000
for i in range(10000):
    for i in range(0, len(x), mini_batch_size):
        mini_batch_x = x[i: i + mini_batch_size].T
        mini_batch_y = y[i: i + mini_batch_size].T

        z1 = l1_w.dot(mini_batch_x) + l1_b
        a1 = relu(z1)
        z2 = l2_w.dot(a1) + l2_b
        a2 = sigmoid(z2)

        cost = np.mean(np.square(a2 - mini_batch_y))
        print(cost)

        da2 = (a2 - mini_batch_y)
        dz2 = da2 * sigmoid_derivative(z2)
        nabla_l2_b = (1/mini_batch_size) * np.sum(dz2, axis=1, keepdims=True)
        nabla_l2_w = (1/mini_batch_size) * np.dot(dz2, a1.T)

        da1 = np.dot(l2_w.T, dz2)
        dz1 = da1 * relu_derivative(a1)
        nabla_l1_b = (1/mini_batch_size) * np.sum(dz1, axis=1, keepdims=True)
        nabla_l1_w = (1/mini_batch_size) * np.dot(dz1, mini_batch_x.T)

        learning_rate = 1

        l1_w = l1_w - learning_rate * nabla_l1_w
        l2_w = l2_w - learning_rate * nabla_l2_w
        l1_b = l1_b - learning_rate * nabla_l1_b
        l2_b = l2_b - learning_rate * nabla_l2_b

    

20.28716753860027
20.28716753851238
20.28716753842451
20.287167538336657
20.287167538248823
20.287167538161004
20.287167538073202
20.287167537985415
20.287167537897645
20.287167537809907


KeyboardInterrupt: 

In [39]:
# check out accuracy
y = dfn[0]

z1 = l1_w.dot(x.T) + l1_b
a1 = relu(z1)
z2 = l2_w.dot(a1) + l2_b
a2 = sigmoid(z2)

count_correct = 0

print(a1.T[0])
print(a2.T[1])

for i in range(len(x)):
    predicted = np.where(a2.T[i]==np.amax(a2.T[i]))[0][0]
    expected = y[i][0]
    if predicted == expected:
        count_correct += 1

print(count_correct)



[0.00000000e+00 5.71181227e-01 8.92892661e-02 2.20493773e-01
 1.13409495e+00 0.00000000e+00 9.75738221e-01 4.82724055e-01
 0.00000000e+00 1.39296050e+00 0.00000000e+00 5.50516696e-01
 2.09571749e-01 0.00000000e+00 0.00000000e+00 2.90848570e-01
 2.38597204e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.67062418e-01 7.78219935e-01 1.95200083e-01 1.87265931e-01
 5.81074144e-04 4.21591584e-01 0.00000000e+00 0.00000000e+00
 4.26906751e-01 5.27145175e-02 0.00000000e+00 1.71380700e+00
 0.00000000e+00 1.69375077e-01 0.00000000e+00 9.58988287e-02
 1.99226501e+00 1.54224467e+00 6.07234664e-01 1.86027197e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.21638904e+00 4.42723528e-02
 6.33513289e-01 0.00000000e+00 1.57791338e-01 1.91743408e-01
 1.06712493e+00 5.44608659e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.17533907e-01 0.00000000e+00
 0.00000000e+00 4.86856158e-01 1.49373625e+00 7.82654347e-02
 0.00000000e+00 0.000000